# An Easy Intro to Deep Q-Learning

Four years ago the nascent AI revolution moved out of image and natural language processing by introducing deep reinforcement learning to the scene. It came through a [paper](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) published in *Nature* by [DeepMind](https://deepmind.com/research/dqn/) showing how a deep neural network was able to surpass human-level performance on many of the Atari games available. It's one network learning to play dozens of games from the raw pixels on the screen. It was a major achievement and was based on **Deep Q-Learning**: [Q-learning](https://www.datahubbs.com/intro-to-q-learning/) with a deep neural network.

## TL;DR
We build a deep Q-learning model with a feed forward network to play OpenAI Gym environments based on the DeepMind algorithm and apply it to `CartPole` for computational ease (the next post will use CNN's to learn directly from pixels).

<figure>
<center>
<img src="https://www.datahubbs.com/wp-content/uploads/2019/02/atari_games.png">
<figcaption>From Mnih et al. *Nature*</figcaption>
</center></figure>

## Deep Q-Networks

Q-learning is predicated upon learning Q-values - i.e. the value of taking a given action when in a given state. Deep Q-Networks (DQN's) are no different in principal from [tabular Q-learning](https://www.datahubbs.com/intro-to-q-learning/), but instead of storing all of our Q-values in a look-up table, we represent them as a neural network. This allows for greater powers of generalization and a richer representation. Take an example from the [Atari game *Breakout*](https://gym.openai.com/envs/#atari), how would you represent the following states in a table?

<img src="https://www.datahubbs.com/wp-content/uploads/2019/02/breakout1.png">
<img src="https://www.datahubbs.com/wp-content/uploads/2019/02/breakout2.png">

Even with high-dimensional arrays, it is not a straightforward process to take these images and transpose them neatly into a state-action array to store Q-values. Moreover, if you were to go about encoding that, how is this going to translate to other games with different state representations such as shown in the DeepMind paper? To get around this, we'll abandon the tabular Q-learning approach and turn to a representation using [deep neural networks](https://www.datahubbs.com/deep-learning-101-first-neural-network-with-pytorch/) and an algorithm known as Deep Q-Learning.

The basics of this approach are almost identical to [Q-learning](https://www.datahubbs.com/intro-to-q-learning/), but the neural network we use in Deep Q-Learning serves as a Q-value function approximation. All this means is we put in the state, and we get our best estimate of the value of taking each action as our output. Think of the neural network as a function that takes states and outputs a number corresponding to how good each action is. That's all it's really doing! Despite this simple function, it's a very powerful tool.

## Target Networks and Experience Replay

Moving to a DQN representation for Q-learning does provide a few difficulties that the tabular versions don't have to deal with. This is caused by the nonlinear deep neural network that we use as a function approximator and can be caused by sequence dependent correlations in the data and frequent updates to your Q approximation. The correlations occur because you move through an episode taking an action $a$ at each time step $t$. The reward you get at time $t$ ($R_t$) is highly correlated to the state and action at $t-1$, which in turn is related to the state and action at $t-2$ and so on. This path dependence makes it difficult to train a DQN. 

This is dealt with using **experience replay**: a memory bank of different states, actions, and rewards that we sample from randomly. This random sampling breaks any sequence dependence in the data when we update the network smoothing out the learning. There's a neuroscience connection here too because something like experience replay is thought to occur in the brain to [encode long-term memories](https://www.sciencedirect.com/science/article/pii/S0166223610000172). 

The second source of instability arises from your estimate of the next state. In tabular Q-learning, you update your values according to the following equation:

$$Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha \big(R_t + \gamma max_a Q(s_{t+1}) - Q(s_t, a_t) \big)$$

With the tabular method, you're only updating $Q(s_t, a_t)$, but with the DQN approach this update changes slightly and the whole network updates at each step. When the whole network updates, your estimate of the best action at the next state ($max_a Q(s_{t+1})$) is changing too. This makes it difficult for the network to learn because the target is always moving meaning the error you're backpropagating isn't consistent from one update to the next for the same states and actions.

To deal with this, DeepMind put in a **target network** which is a copy of the neural network we're training, but we only copy it after every $N$ time steps. This means that the error is going to be stable for some time allowing the network to learn, then the target network gets updated to a better approximation allowing us to begin learning again.

### DQN Loss Function

Because our Q-function is represented by a deep neural network, we need to change the update rule to make it applicable to backpropagation. This **loss function** needs to be differentiable with respect to the network's parameters ($\theta$) as well. The logic for the DQN loss function is essentially the same as for the tabular Q-learning update rule: we take an action according to our Q-function and compare the reward we got with the estimate of our best action in this new state. 

$$\mathcal{L}(\theta) = \bigg(R_t + \gamma max_a \big(Q(s_{t+1}; \theta_t) \big) - Q(s_t, a_t; \theta) \bigg)^2$$

Again, we've got rewards and the Q-value estimates of our states and actions, but now we also have two networks given as $\theta$ and $\theta_t$ being our network and our target network respectively. We're squaring the error here in order to penalize large errors much more than smaller errors. To update the network, we apply the [backpropagation algorithm](https://www.datahubbs.com/deep-learning-101-the-theory/) which takes the derivative of all the values in the layers and makes changes to the corresponding layers. Thankfully, we've got PyTorch to do that for us!

## Deep Q-Learning Algorithm with Experience Replay

We'll be implementing the algorithm as found in the [DeepMind paper](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) (it's in the *Methods* section in case you want to read more on it yourself). Like for any deep reinforcement learning algorithm, we start by initializing our networks and environments, then loop through each episode getting data in the form of states, actions, and rewards. Because Q-learning is on-policy, we'll be updating our policy as we go, but this time using our experience replay buffer to draw samples from. We'll also use an $\epsilon$-greedy selection strategy to choose which action to take next.

> Initialize replay memory $D$ with $M$ samples and select minibatch sample size $B$

> Initialize action-value function $Q$ and target network $Q_t$ with weights $\theta$ and $\theta_t = \theta$

> Select parameters $\alpha, \gamma \in (0, 1]$

> **FOR** each episode:

>> Initialize $s_0$

>>> **FOR** each step $t$ in the episode:

>>>> **IF** $p < \epsilon$ select a random action $a_t$

>>>> **ELSE** select $argmax_a \big(Q(s_t; \theta) \big)$

>>>> Take action $a_t$ and observe reward $R_t$ and new state $s_{t+1}$

>>>> Store transition ($s_t$, $a_t$, $R_t$, $s_{t+1}$) in replay buffer $D$

>>>> Sample random minibatch of $B$ transitions from $D$

>>>> Calculate the loss for all samples: $$\mathcal{L}(\theta) = 
\begin{cases}
  \bigg(R_t + \gamma max_a \big(Q(s_{t+1}; \theta_t) \big) - Q(s_t, a_t; \theta) \bigg)^2 \\    
  \bigg(R_t - Q(s_t, a_t; \theta) \bigg)^2 \quad \text{if } s_{t+1} \text{ is a terminal state}
\end{cases}
$$


>>>> Update parameters $\theta$ with gradient descent

>>>> Every $N$ steps, set $\theta_t = \theta$

This is just like the [Q-learning](https://www.datahubbs.com/intro-to-q-learning/) algorithm except with the aforementioned modifications to accommodate the neural network and to incorporate experience replay. 

## DQN Implementation

To implement everything just like DeepMind did, i.e. a network that is capable of playing most of the Atari-57 suite at or above human level, is going to take a lot of GPU computing resources; resources that many people don't have access to. For that reason, we'll simplify things a bit and only focus on the easier `CartPole` task in this post, and then in the follow-up, we'll add on the convolutional layers up front to learn from the raw pixels themselves. 

I've covered the [basics of `CartPole`](https://www.datahubbs.com/policy-gradients-with-reinforce/) before, so check out the link if you're not familiar with this environment. It's used because it's easy to work with to debug your algorithm and check your understanding before moving onto more challenging environments. 

In case you need to install the Gym environments, it's just:

`pip install gym`

Let's import a few of our basic packages and move on to the code.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import torch
from torch import nn
from collections import namedtuple, deque
from copy import deepcopy, copy

### Deep Q-Network

To start things off, we'll put the DQN together. We'll build our network using the `QNetwork` class which will assemble our network in PyTorch and enable us to get our actions and Q-values with the appropriate methods. All you need to pass here is the specific Gym environment you're working with, and it will be up and running.

In [11]:
class QNetwork(nn.Module):
    
    def __init__(self, env, learning_rate=1e-3, device='cpu'):
        super(QNetwork, self).__init__()
        self.device = device
        self.n_inputs = env.observation_space.shape[0]
        self.n_outputs = env.action_space.n
        self.actions = np.arange(env.action_space.n)
        self.learning_rate = learning_rate
        
        # Set up network
        self.network = nn.Sequential(
            nn.Linear(self.n_inputs, 16, bias=True),
            nn.ReLU(),
            nn.Linear(16, 16, bias=True),
            nn.ReLU(), 
            nn.Linear(16, 16, bias=True),
            nn.ReLU(),
            nn.Linear(16, self.n_outputs, bias=True))
        
        # Set to GPU if cuda is specified
        if self.device == 'cuda':
            self.network.cuda()
            
        self.optimizer = torch.optim.Adam(self.parameters(),
                                          lr=self.learning_rate)
        
    def get_action(self, state, epsilon=0.05):
        if np.random.random() < epsilon:
            action = np.random.choice(self.actions)
        else:
            action = self.get_greedy_action(state)
        return action
    
    def get_greedy_action(self, state):
        qvals = self.get_qvals(state)
        return torch.max(qvals, dim=-1)[1].item()
    
    def get_qvals(self, state):
        if type(state) is tuple:
            state = np.array([np.ravel(s) for s in state])
        state_t = torch.FloatTensor(state).to(device=self.device)
        return self.network(state_t)

The DQN consists of a series of feed-forward layers leading to the output layer which will provide the estimated Q-value. To get a given action out of the network we call the `get_action()` method, which implements our $\epsilon$-greedy strategy. There are also a few helper functions here such as `get_qvals()` and `get_greedy_action()` which return the Q-values of the given state and our estimate of the best action at a given state respectively.

### DQN Memory Bank

For the experience replay piece, we'll construct an `experienceReplayBuffer` to contain our transitions. We can use a `namedtuple` to store the values and load those into a `deque`, which is an object that stores values up to a limit. Once the `deque` limit is crossed, it tosses the first value out to make room for the new transition. This is going to be important for us because we want to keep updating our memory and, as the network learns, we expect newer transitions to be more relevant than older transitions because we should be in better positions. Plus, we don't want to continue growing our replay memory as that will take more and more computing resources as we train.

This class also contains functions for appending values to the buffer and sampling it randomly. The `burn_in` is used to initialize our memory. If the `burn_in` value is set to 10,000, then at the beginning of training, we'll have the network take 10,000 completely random steps to populate the buffer so that we have sufficient values to train on. This will help get a wider variety of data to work with then we'd get if we set the `burn_in` value to 0.

In [4]:
class experienceReplayBuffer:

    def __init__(self, memory_size=50000, burn_in=10000):
        self.memory_size = memory_size
        self.burn_in = burn_in
        self.Buffer = namedtuple('Buffer', 
            field_names=['state', 'action', 'reward', 'done', 'next_state'])
        self.replay_memory = deque(maxlen=memory_size)

    def sample_batch(self, batch_size=32):
        samples = np.random.choice(len(self.replay_memory), batch_size, 
                                   replace=False)
        # Use asterisk operator to unpack deque 
        batch = zip(*[self.replay_memory[i] for i in samples])
        return batch

    def append(self, state, action, reward, done, next_state):
        self.replay_memory.append(
            self.Buffer(state, action, reward, done, next_state))

    def burn_in_capacity(self):
        return len(self.replay_memory) / self.burn_in

Here, we have a few helper functions as well, namely `sample_batch()` and `append()`. `sample_batch()` takes a batch size - let's say 32 in this case - and randomly selects 32 `(state, action, reward, done, next_state)` tuples from the memory and sticks those together into a single batch of 32. This batch is what we'll use to update our network. Of course, we need to add data to the buffer, and that's done using `append()`. The only thing to keep in mind with these is the order we construct our tuples in. `burn_in_capacity()` simply returns the ratio between the data in the buffer and the burn in limit allowing a quick check to see if we've satisfied that basic requirement before training the network.

### Tying it All Together

We need to stick the DQN, replay buffer, and environment together in order to train. For this, we'll create one final class called `DQNAgent` which will take these three pieces as arguments along with a few other parameters, and enable training of the network. I'll refer to this as the **agent** for brevity because it is the object that is going to make decisions using the network and take those actions in the environment. 

Take a look at the code below, and then we'll walk through it for clarity.

In [15]:
class DQNAgent:
    
    def __init__(self, env, network, buffer, epsilon=0.05, batch_size=32):
        
        self.env = env
        self.network = network
        self.target_network = deepcopy(network)
        self.buffer = buffer
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.window = 100
        self.reward_threshold = 195 # Avg reward before CartPole is "solved"
        self.initialize()
    
    def take_step(self, mode='train'):
        if mode == 'explore':
            action = self.env.action_space.sample()
        else:
            action = self.network.get_action(self.s_0, epsilon=self.epsilon)
            self.step_count += 1
        s_1, r, done, _ = self.env.step(action)
        self.rewards += r
        self.buffer.append(self.s_0, action, r, done, s_1)
        self.s_0 = s_1.copy()
        if done:
            self.s_0 = env.reset()
        return done
        
    # Implement DQN training algorithm
    def train(self, gamma=0.99, max_episodes=10000, 
              batch_size=32,
              network_update_frequency=4,
              network_sync_frequency=2000):
        self.gamma = gamma
        # Populate replay buffer
        while self.buffer.burn_in_capacity() < 1:
            self.take_step(mode='explore')
            
        ep = 0
        training = True
        while training:
            self.s_0 = self.env.reset()
            self.rewards = 0
            done = False
            while done == False:
                done = self.take_step(mode='train')
                # Update network
                if self.step_count % network_update_frequency == 0:
                    self.update()
                # Sync networks
                if self.step_count % network_sync_frequency == 0:
                    self.target_network.load_state_dict(
                        self.network.state_dict())
                    self.sync_eps.append(ep)
                    
                if done:
                    ep += 1
                    self.training_rewards.append(self.rewards)
                    self.training_loss.append(np.mean(self.update_loss))
                    self.update_loss = []
                    mean_rewards = np.mean(
                        self.training_rewards[-self.window:])
                    self.mean_training_rewards.append(mean_rewards)
                    print("\rEpisode {:d} Mean Rewards {:.2f}\t\t".format(
                        ep, mean_rewards), end="")
                    
                    if ep >= max_episodes:
                        training = False
                        print('\nEpisode limit reached.')
                        break
                    if mean_rewards >= self.reward_threshold:
                        training = False
                        print('\nEnvironment solved in {} episodes!'.format(
                            ep))
                        break
                        
    def calculate_loss(self, batch):
        states, actions, rewards, dones, next_states = [i for i in batch]
        rewards_t = torch.FloatTensor(rewards).to(device=self.network.device)
        actions_t = torch.LongTensor(np.array(actions)).reshape(-1,1).to(
            device=self.network.device)
        dones_t = torch.ByteTensor(dones).to(device=self.network.device)
        
        qvals = torch.gather(self.network.get_qvals(states), 1, actions_t)
        qvals_next = torch.max(self.target_network.get_qvals(next_states),
                               dim=-1)[0].detach()
        qvals_next[dones_t] = 0 # Zero-out terminal states
        expected_qvals = self.gamma * qvals_next + rewards_t
        loss = nn.MSELoss()(qvals, expected_qvals.reshape(-1,1))
        return loss
    
    def update(self):
        self.network.optimizer.zero_grad()
        batch = self.buffer.sample_batch(batch_size=self.batch_size)
        loss = self.calculate_loss(batch)
        loss.backward()
        self.network.optimizer.step()
        if self.network.device == 'cuda':
            self.update_loss.append(loss.detach().cpu().numpy())
        else:
            self.update_loss.append(loss.detach().numpy())
        
    def initialize(self):
        self.training_rewards = []
        self.training_loss = []
        self.update_loss = []
        self.mean_training_rewards = []
        self.sync_eps = []
        self.rewards = 0
        self.step_count = 0
        self.s_0 = self.env.reset()

The class starts with loading the network and environment as attributes to be called by other methods as well as setting up some empty lists and values to store data and count (see `initialize()`). We will use a `window` value of 100, which just means that we're going to average over the past 100 episodes to guage our progress to be consistent with the [OpenAI performance definitions](https://gym.openai.com/evaluations/eval_NQ6cZZXTR7y4EJQNV6HOA/) which focus on 100-episode averages. For `CartPole-v0`, the environment is considered 'solved' after an average reward of 195 is reached over a 100-episode span. 

The primary method is `train()` which allows us to specify the discount factor `gamma`, the maximum number of episodes, our batch size, how often we update our network, and how regularly we synchronize our target network with out current, trained network. All of these parameters are taken and used to run our training loop where we first populate our `experienceReplayBuffer` with our burn-in length, then let the network take its actions and get feedback from the `experienceReplayBuffer`. 

We call the `take_step()` method both when training and populating the replay buffer. This takes an argument called `mode` which is either set to `train` or `explore`. If we set this to `explore`, we take random actions, so this is used for the burn-in phase. If we set it to `train`, then we recruit the neural network to implement our $\epsilon$-greedy action selection. 

If you keep walking through the training loop, you'll notice that we have our network updating and network synchronization steps next. In the *DeepMind* paper, they set the `network_update_frequency=4`, meaning they only sample from their replay buffer and run the backpropogation algorithm after every four training steps. This is the computationally expensive part of the algorithm, so if we can push that number higher, we'll get an algorithm that will run through more episodes faster, but that doesn't necessarily mean that it will train any faster. 

Synching the network isn't particularly computationally expensive, however, because we use the target network to determine our error, if we update too often, then our network is chasing a moving target. Providing some stability here for a few episodes enables it to minimize the error, then learn the new target. This yields steadier results in the training process. 

Other key methods are `update()` and `calculate_loss()`. We'll look at these in a bit more detail because they are key to training the algorithm and similar functions appear all throughout deep learning work.

First, let's start with `update()`.

In [ ]:
def update(self):
    self.network.optimizer.zero_grad()
    batch = self.buffer.sample_batch(batch_size=self.batch_size)
    loss = self.calculate_loss(batch)
    loss.backward()
    self.network.optimizer.step()
    if self.network.device == 'cuda':
        self.update_loss.append(loss.detach().cpu().numpy())
    else:
        self.update_loss.append(loss.detach().numpy())

I always start my PyTorch `update()` functions with `self.network.optimizer.zero_grad()`. This clears out any gradients in the network optimizer from any possible previous runs allowing us to calculate new gradients going forward. After that, we need to sample a batch from our replay buffer, and then calculate the loss according to our loss function. When we have the `loss`, we differentiate to get the gradients by calling `loss.backward()` and then apply these gradients to the network by calling `self.network.optimizer.step()`. Finally, we store those loss values in the `self.update_loss` list. If we're running it on a GPU system, then we have to send it to the CPU first. But it's always important to call `detach()` on your loss so that PyTorch no longer associates these values with your computational graph.

The other key function here is `calculate_loss()`. It showed up in the `update()` method and returns those key loss values for us to train with.

In [ ]:
def calculate_loss(self, batch):
    states, actions, rewards, dones, next_states = [i for i in batch]
    rewards_t = torch.FloatTensor(rewards).to(device=self.network.device)
    actions_t = torch.LongTensor(np.array(actions)).reshape(-1,1).to(
        device=self.network.device)
    dones_t = torch.ByteTensor(dones).to(device=self.network.device)

    qvals = torch.gather(self.network.get_qvals(states), 1, actions_t)
    qvals_next = torch.max(self.target_network.get_qvals(next_states),
                           dim=-1)[0].detach()
    qvals_next[dones_t] = 0 # Zero-out terminal states
    expected_qvals = self.gamma * qvals_next + rewards_t
    loss = nn.MSELoss()(qvals, expected_qvals.reshape(-1,1))
    return loss

First off, we have to unpack the our batch from the replay memory buffer and seperate these into separate arrays. Then for the rewards, actions, and done values, they need to be converted to PyTorch tensors and shaped properly (the states are converted by the network when we call `get_qvals()`, so we don't need to do that twice). `get_qvals()` returns an array of Q-values with the dimensions `[batch_size, number of actions]` (or $32 \times 2$ in this case). To update our network, we need to get the Q-values for the actions that we actually took, so we use PyTorch's `gather()` function to subset our Q-values appropriately. 

Similarly, because our loss function compares our actual action with the discounted best estimate from the next state $\big(\gamma max_a (Q(s_{t+1}; \theta_t) \big)$, we call `get_qvals()` from our target network and pass the `next_states` array. We use PyTorch's `max()` function to select the maximum of each row (set with `dim=-1`). This function returns a tuple of the maximum value and the index of the maximum value, so we use `[0]` just to get the max value for each row and we call `detach()` to ensure that these values don't update our target network when we call `loss.backward()` and `optimizer.step()`. The `qvals_next` are set to 0 at terminal states because there are no future rewards to discount. Then we combine all of these together with the reward and the discount factor to get the `expected_qvals`. Finally, the loss is calculated as the mean squared error of the Q-values of the actions we took minus the expected, discounted Q-values from our target network and the loss is returned to the update function.

There's a lot there, but all of it was just to calculate the loss according to:

$$\mathcal{L}(\theta) = 
\begin{cases}
  \bigg(R_t + \gamma max_a \big(Q(s_{t+1}; \theta_t) \big) - Q(s_t, a_t; \theta) \bigg)^2 \\    
  \bigg(R_t - Q(s_t, a_t; \theta) \bigg)^2 \quad \text{if } s_{t+1} \text{ is a terminal state}
\end{cases}
$$

With that, I think we're finally ready to train the network! We do that by initializing the environment, buffer, DQN, and the agent, then calling `agent.train()`. 

In [20]:
env = gym.make('CartPole-v0')
buffer = experienceReplayBuffer(memory_size=10000, burn_in=1000)
dqn = QNetwork(env, learning_rate=1e-3)
agent = DQNAgent(env, dqn, buffer)
agent.train(max_episodes=5000, network_update_frequency=1, 
            network_sync_frequency=2500)

Episode 1396 Mean Rewards 195.10		
Environment solved in 1396 episodes!


And plotting the rewards:

<img src="https://www.datahubbs.com/wp-content/uploads/2019/03/dqn_cartpole_training.png">

Hopefully this will train fairly quickly for you on a CPU. It may take a few tries and different hyperparameters to get it to train quickly, but it should work and be easy to de-bug. 

I pulled out the basics from the *DeepMind* paper on Deep Q-Learning, but there are a few more things to get into to see how their model worked and to be able to apply it to learning from raw pixels. I'll get into that next time where we'll add a few convolutional neural network layers to the front-end of our DQN to enable it to learn from the images we pass. In the meantime, if you want to play around with the Atari games, there are RAM versions which read data directly from the system and represent the game state as a vector of 128 numbers. You'll likely need a larger neural network, but the tutorial here will let you plug one of those in as your environment and let you play Atari now. Just use `pip install gym[atari]` and set your environment to something like `Breakout-ram-v0` and give it a shot!

Also, if you want to get some more versatile code that you can run from the command line, check out the `dqn.py` file [here on GitHub](https://github.com/hubbs5/rl_blog/blob/master/q_learning/deep/dqn.py).